In [5]:
import logging
import os
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR
from sqlalchemy import inspect
import datetime


In [12]:
db_host = "192.168.50.60"  # 数据库服务主机
db_user = "root"  # 数据库访问用户
db_password = "mariadb"  # 数据库访问密码
db_database = "instockdb"  # 数据库名称
db_port = 3306  # 数据库服务端口
db_charset = "utf8mb4"  # 数据库字符集

# 使用环境变量获得数据库,docker -e 传递
_db_host = os.environ.get('db_host')
if _db_host is not None:
    db_host = _db_host
_db_user = os.environ.get('db_user')
if _db_user is not None:
    db_user = _db_user
_db_password = os.environ.get('db_password')
if _db_password is not None:
    db_password = _db_password
_db_database = os.environ.get('db_database')
if _db_database is not None:
    db_database = _db_database
_db_port = os.environ.get('db_port')
if _db_port is not None:
    db_port = int(_db_port)

MYSQL_CONN_URL = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    db_user, db_password, db_host, db_port, db_database, db_charset)

MYSQL_CONN_DBAPI = {'host': db_host, 'user': db_user, 'password': db_password, 'database': db_database,
                    'charset': db_charset, 'port': db_port, 'autocommit': True}


In [13]:
def get_today(sep='-'):
    TODAY = datetime.date.today()
    fstr = "%Y" + sep + "%m" + sep + "%d"
    today = TODAY.strftime(fstr)
    return today

# 前一日
def day_last_days(daynow, last=-1):
    return str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() + datetime.timedelta(last))

def day_duration_day(daynow, lastday):
    datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date()
# 上一个工作日
def last_tddate(days=1):
    # today = datetime.datetime.today().date() + datetime.timedelta(-days)
    if days is None:
        return days
    today = datetime.datetime.today().date()
    # log.debug("today:%s " % (today))
    # return str(today)

    def get_work_day(today):
        day_n = int(today.strftime("%w"))
        if day_n == 0:
            lastd = today + datetime.timedelta(-2)
            # log.debug("0:%s" % lastd)
        elif day_n == 1:
            lastd = today + datetime.timedelta(-3)
            # log.debug("1:%s" % lastd)
        else:
            lastd = today + datetime.timedelta(-1)
            # log.debug("2-6:%s" % lastd)
        return lastd
        # if days==0:
        # return str(lasd)
    lastday = today
    for x in range(int(days)):
        # print x
        lastday = get_work_day(today)
        today = lastday
    return str(lastday)

# 通过数据库链接 engine
def engine():
    return create_engine(MYSQL_CONN_URL)


def engine_to_db(to_db):
    _engine = create_engine(MYSQL_CONN_URL.replace(f'/{db_database}?', f'/{to_db}?'))
    return _engine


# DB Api -数据库连接对象connection，有游标
def conn_with_cursor():
    return conn_not_cursor().cursor()

# DB Api -数据库连接对象connection，无游标
def conn_not_cursor():
    try:
        _db = pymysql.connect(**MYSQL_CONN_DBAPI)
    except Exception as e:
        logging.error(f"database.conn_not_cursor处理异常：{MYSQL_CONN_DBAPI}{e}")
    return _db

# 查询数据
def executeSqlFetch(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.executeSqlFetch处理异常：{sql}{e}")

        result = db.fetchall()
        db.close()
        return result

# 计算数量
def executeSqlCount(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.select_count计算数量处理异常：{e}")

        result = db.fetchall()
        db.close()
        # 只有一个数组中的第一个数据
        if len(result) == 1:
            return int(result[0][0])
        else:
            return 0


In [14]:
try:
    with pymysql.connect(**MYSQL_CONN_DBAPI) as mydb:
        mydb.cursor().execute(" select 1 ")
except Exception as e:
    logging.error("执行信息：数据库不存在，将创建。")


1

In [15]:
import pandas as pd
def panda_df(conn, sql):
    df = pd.read_sql(sql, conn)
    return df


from pandas import DataFrame as df
def exe_sql_select(conn, sql):
    cur = conn.cursor()
    # cur = conn
    try:  # 使用异常处理，以防程序无法正常运行
        cur.execute(sql)
        data = df(cur.fetchall(), columns=[col[0] for col in cur.description])
    except Exception as e:
        conn.rollback()  # 发生错误时回滚
        print('事务处理失败', e)
    else:
        # conn.commit() # 事务提交
        print('事务处理成功', cur.rowcount)
    # cur.close()
    return data


conn = pymysql.connect(**MYSQL_CONN_DBAPI)

_selcol = '*'
_table_name = 'cn_stock_strategy_enter'
date = last_tddate(1)
# today_now = get_today()
today_now = last_tddate(0)

_selcol = f''' `code`,`name`,`rate_1` '''
sql = f'''SELECT '{_selcol}' FROM `{_table_name}` WHERE `date` = '{date}' and 
								`kdjk` >= 80 and `kdjd` >= 70 and `kdjj` >= 100 and `rsi_6` >= 80 and 
								`cci` >= 100 and `cr` >= 300 and `wr_6` >= -20 and `vr` >= 160'''

sql_now_price = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{date}' AND rate_1 > 0 '''
sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''
sql_tables_name =f'''select table_name from information_schema.tables where table_schema=`{db_database}` '''
# sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where TABLE_SCHEMA=`{db_database}` and table_name = `{_table_name}`'''

sql_tables_name ='SHOW DATABASES'
# print('当前库下所有表的名称')
# sql_columns = "select COLUMN_NAME from information_schema.COLUMNS where table_name = '%s'"%(_table_name)
sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where table_name = '{_table_name}' '''
print(sql_now_price)
mycursor = conn.cursor()
mycursor.execute(sql_now_price)
print(_selcol)
count = 0
for field in mycursor.fetchall():
    # print(field[0],end='  ')
    # print(field[1:4])
    # print(field)
    count +=1
print("all:%s" %(count))

df = exe_sql_select(conn, sql_now_price)

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(conn, sql_now_today)


# mycursor.execute(sql_columns)
# mycursor.execute(sql_now)

# myresult = mycursor.fetchall()
# for x in myresult:
#   print(x)

conn.close()


SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2023-05-10' AND rate_1 > 0 


15

 `code`,`name`,`rate_1` 
all:15
事务处理成功 15


c:\Users\Johnson\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\20200530'
code = '000002'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")


Y:\dockerf\stock\instockcache\hist\202005\20200530\000002qfq.gzip.pickle


In [8]:
df.shape
df[-5:]
df[:1]
# df.columns
lastday=df.date.values[-1]
lastday


(728, 11)

,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
723,2023-05-24,14.47,14.11,14.48,14.10,752758,1.068559e+09,2.62,-2.82,-0.41,0.77
724,2023-05-25,14.03,14.19,14.19,14.00,423953,5.969041e+08,1.35,0.57,0.08,0.44
725,2023-05-26,14.14,14.21,14.24,14.01,407754,5.767348e+08,1.62,0.14,0.02,0.42
726,2023-05-29,14.17,14.03,14.18,14.00,443019,6.224551e+08,1.27,-1.27,-0.18,0.46
727,2023-05-30,13.99,14.36,14.36,13.67,881969,1.230122e+09,4.92,2.35,0.33,0.91


,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
0,2020-06-01,22.74,23.16,23.26,22.74,779004,2.049366e+09,2.32,3.12,0.7,0.8


'2023-05-30'

In [3]:
# from sympy import true
def get_today_duration(startday, endday=None):
    if startday is not None and len(startday) > 6:
        if endday:
            today = datetime.datetime.strptime((endday), '%Y-%m-%d').date()
        else:
            today = datetime.date.today()
        # if get_os_system() == 'mac':
        #     # last_day = datetime.datetime.strptime(datastr, '%Y/%m/%d').date()
        #     last_day = datetime.datetime.strptime(datastr, '%Y-%m-%d').date()
        # else:
        #     # last_day = datetime.datetime.strptime(datastr, '%Y/%m/%d').date()
        #     last_day = datetime.datetime.strptime(datastr, '%Y-%m-%d').date()
        last_day = datetime.datetime.strptime(startday, '%Y-%m-%d').date()
        
        duration_day = int((today - last_day).days)
    else:
        duration_day = None
    return (duration_day)


def get_tdx_stock_period_to_type(stock_data, period_day='W-FRI', periods=5, ncol=None, ratiodays=True):
    """_周期转换周K,月K_

    Returns:
        _type_: _description_
    """
    #快速日期处理
    #https://www.likecs.com/show-204682607.html

    period_type = period_day
    if 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
    stock_data['date'] = stock_data.index
    lastday = stock_data.date.values[-1]
    lastday2 = stock_data.date.values[-2]
    duration_day = get_today_duration(lastday2,lastday)
    # print("duration:%s"%(duration_day))
    if duration_day > 3:
        if 'date' in stock_data.columns:
            stock_data = stock_data.drop(['date'], axis=1)
        return stock_data.reset_index()
    indextype = True if stock_data.index.dtype == 'datetime64[ns]' else False
    # if cct.get_work_day_status() and 915 < cct.get_now_time_int() < 1500:
    #     stock_data = stock_data[stock_data.index < cct.get_today()]

    if stock_data.index.name == 'date':
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    elif 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
        stock_data.sort_index(ascending=True, inplace=True)
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    # else:
    #     log.error("index.name not date,pls check:%s" % (stock_data[:1]))

    period_stock_data = stock_data.resample(period_type).last()
    # period_stock_data['percent']=stock_data['percent'].resample(period_type,how=lambda x:(x+1.0).prod()-1.0)
    # print stock_data.index[0],stock_data.index[-1]
    # period_stock_data.index =
    # pd.DatetimeIndex(start=stock_data.index.values[0],end=stock_data.index.values[-1],freq='BM')

    period_stock_data['open'] = stock_data[
        'open'].resample(period_type).last()
    period_stock_data['high'] = stock_data[
        'high'].resample(period_type).max()
    period_stock_data['low'] = stock_data[
        'low'].resample(period_type).min()

    if ratiodays:
        daynow = str(period_stock_data['open'].index.values[-1])[:10]
        # daynow = '2023-04-26'
        # lastday = '2023-04-23'
        duratio = int(str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])
        ratio_d =(5-(duratio%5))/5
        print("ratio_d:%s %s"%(ratio_d,lastday))
    else:
        ratio_d = 1
        print(ratio_d)
        
    if ncol is not None:
        for co in ncol:
            period_stock_data[co] = stock_data[co].resample(period_type).sum()
            if ratiodays:
                period_stock_data[co] = period_stock_data[co].apply(lambda x: round(x / ratio_d, 1))
                
    # else:
    period_stock_data['amount'] = stock_data[
        'amount'].resample(period_type).sum()
    period_stock_data['volume'] = stock_data[
        'volume'].resample(period_type).sum()
    if ratiodays:
        period_stock_data['amount'] = period_stock_data['amount'].apply(lambda x: round(x / ratio_d, 1))
        period_stock_data['volume'] = period_stock_data['volume'].apply(lambda x: round(x / ratio_d, 1))
                
    # period_stock_data['turnover']=period_stock_data['vol']/(period_stock_data['traded_market_value'])/period_stock_data['close']
    period_stock_data.index = stock_data['date'].resample(period_type).last().index
    # print period_stock_data.index[:1]
    if 'code' in period_stock_data.columns:
        period_stock_data = period_stock_data[period_stock_data['code'].notnull()]
    period_stock_data = period_stock_data.dropna()
    # period_stock_data.reset_index(inplace=True)
    # period_stock_data.set_index('date',inplace=True)
    # print period_stock_data.columns,period_stock_data.index.name
    # and period_stock_data.index.dtype != 'datetime64[ns]')
    
    # if not indextype and period_stock_data.index.name == 'date':
    #     # stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    #     period_stock_data.index = [str(x)[:10] for x in period_stock_data.index]
    #     period_stock_data.index.name = 'date'
    # else:
    #     if 'date' in period_stock_data.columns:
    #         period_stock_data = period_stock_data.drop(['date'], axis=1)
    
    if 'date' in period_stock_data.columns:
            period_stock_data = period_stock_data.drop(['date'], axis=1)
    return period_stock_data.reset_index()


In [4]:
print(get_today_duration('2023-04-28','2023-05-01'))

NameError: name 'datetime' is not defined

In [9]:
dfw = get_tdx_stock_period_to_type(df.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
dfw[-3:]
dfw = get_tdx_stock_period_to_type(df.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-4:-1]


1


,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
151,2023-05-12,15.24,15.06,15.89,15.06,3865669,5.960572e+09,11.11,-3.12,-0.24,3.98
152,2023-05-19,14.62,14.54,15.12,14.50,2509911,3.711475e+09,8.32,-3.50,-0.08,2.58
153,2023-05-26,14.47,14.11,14.76,14.10,1502345,2.162481e+09,6.05,-2.95,-0.41,1.55


ratio_d:0.6 2023-05-24


,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
150,2023-05-05,14.95,15.54,15.64,14.72,3701580.0,5.634961e+09,12.9,4.5,0.59,3.8
151,2023-05-12,15.24,15.06,15.89,15.06,6442781.7,9.934287e+09,18.5,-5.2,-0.24,6.6
152,2023-05-19,14.62,14.54,15.12,14.50,4183185.0,6.185792e+09,13.9,-5.8,-0.08,4.3


In [21]:
pw_close_max3 = dfw.iloc[-4:-1]['close'].max()
pw_close_max3


15.54

In [24]:
daynow=dfw.date.values[-1]
# str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0]
# int(str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])


## 新浪龙虎榜

In [25]:
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

def _find_last_page(url: str = None, recent_day: str = "60"):
    url = "http://vip.stock.finance.sina.com.cn/q/go.php/vLHBData/kind/ggtj/index.phtml"
    params = {
        "last": recent_day,
        "p": "1",
    }
    r = requests.get(url, params=params)
    soup = BeautifulSoup(r.text, "lxml")
    try:
        previous_page = int(soup.find_all(attrs={"class": "page"})[-2].text)
    except:
        previous_page = 1
    if previous_page != 1:
        while True:
            params = {
                "last": recent_day,
                "p": previous_page,
            }
            r = requests.get(url, params=params)
            soup = BeautifulSoup(r.text, "lxml")
            last_page = int(soup.find_all(attrs={"class": "page"})[-2].text)
            if last_page != previous_page:
                previous_page = last_page
                continue
            else:
                break
    return previous_page

def stock_lhb_ggtj_sina(recent_day: str = "30") -> pd.DataFrame:
    """
    龙虎榜-个股上榜统计
    http://vip.stock.finance.sina.com.cn/q/go.php/vLHBData/kind/ggtj/index.phtml
    :param recent_day: choice of {"5": 最近 5 天; "10": 最近 10 天; "30": 最近 30 天; "60": 最近 60 天;}
    :type recent_day: str
    :return: 龙虎榜-每日详情
    :rtype: pandas.DataFrame
    """
    url = "http://vip.stock.finance.sina.com.cn/q/go.php/vLHBData/kind/ggtj/index.phtml"
    last_page_num = _find_last_page(url, recent_day)
    big_df = pd.DataFrame()
    for page in tqdm(range(1, last_page_num + 1), leave=False):
        params = {
            "last": recent_day,
            "p": page,
        }
        r = requests.get(url, params=params)
        temp_df = pd.read_html(r.text)[0].iloc[0:, :]
        big_df = pd.concat([big_df, temp_df], ignore_index=True)
    big_df["股票代码"] = big_df["股票代码"].astype(str).str.zfill(6)
    big_df.columns = ["股票代码", "股票名称", "上榜次数", "累积购买额", "累积卖出额", "净额", "买入席位数", "卖出席位数"]
    return big_df
# data = stock_lhb_ggtj_sina(recent_day="5")

from sqlalchemy import DATE, NVARCHAR, FLOAT, SmallInteger
TABLE_CN_STOCK_TOP = {'name': 'cn_stock_top', 'cn': '股票龙虎榜',
                      'columns': {'date': {'type': DATE, 'cn': '日期'}, 'code': {'type': NVARCHAR(6), 'cn': '代码'},
                                  'name': {'type': NVARCHAR(20), 'cn': '名称'},
                                  'ranking_times': {'type': FLOAT, 'cn': '上榜次数'},
                                  'sum_buy': {'type': FLOAT, 'cn': '累积购买额'},
                                  'sum_sell': {'type': FLOAT, 'cn': '累积卖出额'},
                                  'net_amount': {'type': FLOAT, 'cn': '净额'},
                                  'buy_seat': {'type': FLOAT, 'cn': '买入席位数'},
                                  'sell_seat': {'type': FLOAT, 'cn': '卖出席位数'}}}

def is_a_stock(code):
    # 上证A股  # 深证A股
    return code.startswith(('600', '601', '603', '605', '000', '001', '002', '300', '301'))

def fetch_stock_top_data(date):
    try:
        data = stock_lhb_ggtj_sina(recent_day="5")
        if data is None or len(data.index) == 0:
            print("data is None")
            return None
        _columns = list(TABLE_CN_STOCK_TOP['columns'])
        _columns.pop(0)
        data.columns = _columns
        data = data.loc[data['code'].apply(is_a_stock)]
        print(data.shape)
        # print(data.drop_duplicates('code'))
        if date is None:
            data.insert(0, 'date', datetime.datetime.now().strftime("%Y-%m-%d"))
        else:
            data.insert(0, 'date', date.strftime("%Y-%m-%d"))
        return data
    except Exception as e:
        logging.error(f"stockfetch.fetch_stock_top_data处理异常：{e}")
    return None

data = (fetch_stock_top_data(None))

(245, 8)


In [26]:
# data = fetch_stock_top_data(None)
data.date[0]

'2023-05-11'

In [27]:
# data = stock_lhb_ggtj_sina(recent_day="5")
# data.drop_duplicates('股票代码')

import pymysql
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR
from sqlalchemy import inspect

# DB Api -数据库连接对象connection，有游标
def conn_with_cursor():
    return conn_not_cursor().cursor()

# DB Api -数据库连接对象connection，无游标
def conn_not_cursor():
    try:
        _db = pymysql.connect(**MYSQL_CONN_DBAPI)
    except Exception as e:
        logging.error(f"database.conn_not_cursor处理异常：{MYSQL_CONN_DBAPI}{e}")
    return _db

def checkTableIsExist(tableName):
    with conn_with_cursor() as db:
        db.execute("""
            SELECT COUNT(*)
            FROM information_schema.tables
            WHERE table_name = '{0}'
            """.format(tableName.replace('\'', '\'\'')))

    if db.fetchone()[0] == 1:
        db.close()
        return True

    db.close()
    return False

# 增删改数据
def executeSql(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
            db.close()
        except Exception as e:
            logging.error(f"database.executeSql处理异常：{sql}{e}")
            
def get_field_types(cols):
    data = {}
    for k in cols:
        data[k] = cols[k]['type']
    return data

# 定义通用方法函数，插入数据库表，并创建数据库主键，保证重跑数据的时候索引唯一。
def insert_db_from_df(data, table_name, cols_type, write_index, primary_keys):
    # 插入默认的数据库。
    insert_other_db_from_df(None, data, table_name, cols_type, write_index, primary_keys)


# 增加一个插入到其他数据库的方法。
def insert_other_db_from_df(to_db, data, table_name, cols_type, write_index, primary_keys):
    # 定义engine
    if to_db is None:
        engine_mysql = engine()
    else:
        engine_mysql = engine_to_db(to_db)
    # 使用 http://docs.sqlalchemy.org/en/latest/core/reflection.html
    # 使用检查检查数据库表是否有主键。
    ipt = inspect(engine_mysql)
    col_name_list = data.columns.tolist()
    # 如果有索引，把索引增加到varchar上面。
    if write_index:
        # 插入到第一个位置：
        col_name_list.insert(0, data.index.name)
    try:
        if cols_type is None:
            data.to_sql(name=table_name, con=engine_mysql, schema=to_db, if_exists='append',
                        index=write_index, )
        elif not cols_type:
            data.to_sql(name=table_name, con=engine_mysql, schema=to_db, if_exists='append',
                        dtype={col_name: NVARCHAR(255) for col_name in col_name_list}, index=write_index, )
        else:
            data.to_sql(name=table_name, con=engine_mysql, schema=to_db, if_exists='append',
                        dtype=cols_type, index=write_index, )
    except Exception as e:
        logging.error(f"database.insert_other_db_from_df处理异常：{table_name}表{e}")

    # 判断是否存在主键
    if not ipt.get_pk_constraint(table_name)['constrained_columns']:
        try:
            # 执行数据库插入数据。
            conn_with_cursor().execute(f'ALTER TABLE `{table_name}` ADD PRIMARY KEY ({primary_keys});')
        except Exception as e:
            logging.error(f"database.insert_other_db_from_df处理异常：{table_name}表{e}")
            
def save_nph_stock_top_data(date, before=True):
    if before:
        return

    try:
        data = fetch_stock_top_data(date)
        if data is None or len(data.index) == 0:
            return
        # print(data.shape)
        table_name = TABLE_CN_STOCK_TOP['name']
        # print("table_name:%s"%(table_name))
        # 删除老数据。
        if checkTableIsExist(table_name):
            if date != data.date[0]:
                # date= datetime.datetime.now().strftime("%Y-%m-%d")
                date= data.date[0]
            del_sql = f"DELETE FROM `{table_name}` where `date` = '{date}'"
            print("checkTableIsExist: sql:%s"%(del_sql))
            executeSql(del_sql)
            cols_type = None
        else:
            cols_type = get_field_types(TABLE_CN_STOCK_TOP['columns'])
        # print(data.shape,data[:2])
        insert_db_from_df(data, table_name, cols_type, False, "`date`,`code`")
    except Exception as e:
        logging.error(f"basic_data_other_daily_job.save_stock_top_data处理异常：{e}")
        

In [28]:
save_nph_stock_top_data(None,before=False)

(245, 8)
checkTableIsExist: sql:DELETE FROM `cn_stock_top` where `date` = '2023-05-11'
